In [1]:
from ros_robot_controller_msgs.srv import GetBusServoState
from ros_robot_controller_msgs.msg import GetBusServoCmd, BusServoState
import rclpy
from rclpy.node import Node
from rclpy.callback_groups import ReentrantCallbackGroup, MutuallyExclusiveCallbackGroup
from cv_bridge import CvBridge
from sensor_msgs.msg import Image
import numpy as np
import cv2
import json


In [2]:
class ImageProcessor:
    def __init__(self, node):
        self.node = node
        self.bridge = CvBridge()
        self.template_file = '/home/ubuntu/ros2_ws/src/poc_pkg/poc_pkg/template.png'
        self.camera_type = 'Mono'
        self.camera = 'usb_cam'
        calibration_file = "/home/ubuntu/ros2_ws/src/poc_pkg/poc_pkg/camera_calibration_data.json"
        with open(calibration_file, "r") as f:
            calibration_data = json.load(f)
        # Инициализация ORB детектора
        # стало:
        if hasattr(cv2, "SIFT_create"):
            self.sift = cv2.SIFT_create(
                nfeatures=3000,         # больше точек → устойчивее гомография
                contrastThreshold=0.05, # не слишком высоко, чтобы не терять слабые
                edgeThreshold=10,
                sigma=1.6
            )
        else:
            # fallback, если в сборке нет SIFT (редко, но бывает)
            self.node.get_logger().warn("SIFT недоступен, переключаюсь на AKAZE")
            self.sift = cv2.AKAZE_create()  # тоже норм для текста


        self.camera_matrix = np.array(calibration_data["camera_matrix"])
        self.dist_coeff = np.array(calibration_data["dist_coeff"])
        self.image_cb_group = MutuallyExclusiveCallbackGroup()
        self.tvec = None


        self.node.create_subscription(Image, '/%s/image_raw' % self.camera,
                                self.process_image, 1, callback_group=self.image_cb_group)
    
    def pixel_to_cam_XY(self, u, v, K_used, Zc):
        fx, fy = K_used[0,0], K_used[1,1]
        cx, cy = K_used[0,2], K_used[1,2]
        x_n = (u - cx) / fx
        y_n = (v - cy) / fy
        X = x_n * Zc
        Y = y_n * Zc
        return float(X), float(Y)  # Z = Zc

    def __is_reasonable_quad(self, quad, img_w, img_h,
                       angle_tol=20,   # ° допуск к 90°
                       len_tol=0.4):   # 40 % разброс противоп. сторон
        """quad – (4,2) numpy: 0‑TL,1‑BL,2‑BR,3‑TR."""
        # 0. все точки в кадре?
        if (quad[:,0] < 0).any() or (quad[:,1] < 0).any() \
        or (quad[:,0] > img_w).any() or (quad[:,1] > img_h).any():
            return False

        # 1. длины рёбер
        edges   = np.vstack((quad[1:], quad[:1])) - quad  # 4×2
        lengths = np.linalg.norm(edges, axis=1)
        if not (abs(lengths[0]-lengths[2]) < len_tol*lengths[0] and
                abs(lengths[1]-lengths[3]) < len_tol*lengths[1]):
            return False

        # 2. углы ~90°
        for i in range(4):
            a, b = edges[i], edges[(i+1) % 4]
            cos = np.dot(a, b) / (np.linalg.norm(a)*np.linalg.norm(b))
            angle = np.degrees(np.arccos(np.clip(cos, -1, 1)))
            if abs(angle-90) > angle_tol:
                return False
        return True

    def save_image(self, file_path):
        """Сохраняет последнее обработанное изображение."""
        cv2.imwrite(file_path, self.undistorted_image) 

    def process_image(self, msg):
        cv_image = self.bridge.imgmsg_to_cv2(msg, "bgr8")
        h, w = cv_image.shape[:2]
        self.undistorted_image = cv2.undistort(cv_image, self.camera_matrix, self.dist_coeff)
        
        cv2.imshow("Undistorted Image", self.undistorted_image)
        cv2.waitKey(1)
        # --- шаблон читаем в цвете и в градациях ---
        tpl_bgr = cv2.imread(self.template_file, cv2.IMREAD_COLOR)
        if tpl_bgr is None:
            self.node.get_logger().error("Template image not found.")
            return
        template = cv2.cvtColor(tpl_bgr, cv2.COLOR_BGR2GRAY)

        scene_bgr = self.undistorted_image
        gray_scene = cv2.cvtColor(scene_bgr, cv2.COLOR_BGR2GRAY)

        # --- лёгкое выравнивание контраста (лучше, чем blur для SIFT/текста) ---
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        template = clahe.apply(template)
        gray_scene = clahe.apply(gray_scene)


        # --- SIFT дескрипторы (L2‑метрика) ---
        kp_template, des_template = self.sift.detectAndCompute(template, None)
        kp_scene,    des_scene    = self.sift.detectAndCompute(gray_scene, None)

        if des_template is None or des_scene is None or len(kp_scene) < 20:
            self.node.get_logger().info('Нет достаточных дескрипторов – пропускаю кадр')
            pass

        # --- FLANN (KD‑деревья) для SIFT ---
        FLANN_INDEX_KDTREE = 1
        index_params  = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
        search_params = dict(checks=64)
        flann = cv2.FlannBasedMatcher(index_params, search_params)

        # 2‑NN матчи в обе стороны (для симметрии)
        m12 = flann.knnMatch(des_template, des_scene, k=2)
        m21 = flann.knnMatch(des_scene, des_template, k=2)

        ratio = 0.6   # типично 0.7–0.8 для SIFT
        good_12 = []
        for m, n in m12:
            if m.distance < ratio * n.distance:
                good_12.append(m)

        # симметричная проверка (оставляем only mutual best)
        rev = {}
        for m, n in m21:
            if m.distance < ratio * n.distance:
                rev[m.queryIdx] = m.trainIdx    # scene->template

        good = []
        for m in good_12:
            if rev.get(m.trainIdx, -1) == m.queryIdx:
                good.append(m)

        self.node.get_logger().info(
            f'KPs: tpl={len(kp_template)} scene={len(kp_scene)}; good={len(good)}'
        )

        if len(good) < 12:
            self.node.get_logger().info('Мало хороших матчей')
            return
        # --- точки для гомографии ---
        src_pts = np.float32([kp_template[m.queryIdx].pt for m in good]).reshape(-1,1,2)
        dst_pts = np.float32([kp_scene[m.trainIdx].pt    for m in good]).reshape(-1,1,2)

        # --- робастная гомография (USAC_MAGSAC, если доступен) ---
        M, inl = cv2.findHomography(
            src_pts, dst_pts, cv2.RANSAC,
            ransacReprojThreshold=3.0, maxIters=5000, confidence=0.999
        )
        if M is None:
            self.node.get_logger().info('Гомография не найдена')
            return


        # --- проецируем прямоугольник шаблона ---
        h, w = template.shape
        pts_template = np.float32([[0,0],[0,h-1],[w-1,h-1],[w-1,0]]).reshape(-1,1,2)
        pts_scene = cv2.perspectiveTransform(pts_template, M)
        quad = pts_scene.reshape(4,2)

        # рисуем полигон
        cv2.polylines(self.undistorted_image, [np.int32(pts_scene)], True, (0,255,0), 3, cv2.LINE_AA)
        # Центр прямоугольника
        center = np.mean(quad, axis=0).astype(int)
        cv2.circle(self.undistorted_image, tuple(center), 5, (0, 0, 255), -1)

        X, Y = self.pixel_to_cam_XY(center[0], center[1], self.camera_matrix, 0.15)

        offset_x, offset_y = -0.01, 0.02

        x_err = offset_x - X
        y_err = offset_y - Y

        cv2.putText(self.undistorted_image, f'X, Y: {X, Y}', (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        cv2.putText(self.undistorted_image, f'X err, Y err: {x_err, y_err}', (10, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        cv2.imshow("Object", self.undistorted_image)
        cv2.waitKey(1)


In [3]:

class TestCamNode(Node):
    def __init__(self):
        super().__init__('test_node')
        self.image_processor = ImageProcessor(self)

        self.get_logger().info('TestNode has been started.')

        



    def call_service(self):
        request = GetBusServoState.Request()
        request.cmd = [GetBusServoCmd(id=i, get_position=1) for i in [1, 2, 3, 4, 5, 10]]
        future = self.client.call_async(request)
        future.add_done_callback(self.cb)

    def cb(self, future):
        try:
            response = future.result()
            print(response)
            self.get_logger().info(f'Servo states: {response.state}')
        except Exception as e:
            self.get_logger().error(f'Service call failed: {e}')

In [4]:
rclpy.init()
node = TestCamNode()

# Используем многопоточный executor, но благодаря MutuallyExclusiveCallbackGroup обработка callback-ов будет последовательной.
executor = rclpy.executors.MultiThreadedExecutor()
executor.add_node(node)
try:
    executor.spin()
except KeyboardInterrupt:
    node.get_logger().info("Останавливаем выполнение...")
    node.image_processor.save_image("/home/ubuntu/ros2_ws/src/poc_pkg/poc_pkg/last_image.jpg")
finally:
    node.destroy_node()
    rclpy.shutdown()

[INFO] [1754900713.352605028] [test_node]: TestNode has been started.
[INFO] [1754900753.518565819] [test_node]: KPs: tpl=508 scene=3000; good=19
[INFO] [1754900753.893561094] [test_node]: KPs: tpl=508 scene=3000; good=16
[INFO] [1754900754.251193386] [test_node]: KPs: tpl=508 scene=3000; good=17
[INFO] [1754900754.631105803] [test_node]: KPs: tpl=508 scene=3000; good=17
[INFO] [1754900755.010110220] [test_node]: KPs: tpl=508 scene=3000; good=17
[INFO] [1754900755.385601667] [test_node]: KPs: tpl=508 scene=3000; good=16
[INFO] [1754900755.783670206] [test_node]: KPs: tpl=508 scene=3000; good=16
[INFO] [1754900756.134708503] [test_node]: KPs: tpl=508 scene=3000; good=17
[INFO] [1754900756.490846086] [test_node]: KPs: tpl=508 scene=3000; good=23
[INFO] [1754900756.860850384] [test_node]: KPs: tpl=508 scene=3000; good=20
[INFO] [1754900757.236155127] [test_node]: KPs: tpl=508 scene=3000; good=16
[INFO] [1754900757.584274689] [test_node]: KPs: tpl=508 scene=3000; good=25
[INFO] [1754900757

RCLError: failed to shutdown: rcl_shutdown already called on the given context, at ./src/rcl/init.c:241

In [ ]:
class TestNode(Node):
    def __init__(self):
        super().__init__('test_node')
        self.client = self.create_client(GetBusServoState, 'ros_robot_controller/bus_servo/get_state')
        self.client.wait_for_service()
        self.get_logger().info('TestNode has been started.')

        self.call_service()



    def call_service(self):
        request = GetBusServoState.Request()
        request.cmd = [GetBusServoCmd(id=i, get_position=1) for i in [1, 2, 3, 4, 5, 10]]
        future = self.client.call_async(request)
        future.add_done_callback(self.cb)

    def cb(self, future):
        try:
            response = future.result()
            print(response)
            self.get_logger().info(f'Servo states: {response.state}')
        except Exception as e:
            self.get_logger().error(f'Service call failed: {e}')

In [ ]:
rclpy.init()
node = TestNode()

# Используем многопоточный executor, но благодаря MutuallyExclusiveCallbackGroup обработка callback-ов будет последовательной.
executor = rclpy.executors.MultiThreadedExecutor()
executor.add_node(node)
try:
    executor.spin()
except KeyboardInterrupt:
    node.get_logger().info("Останавливаем выполнение...")
finally:
    node.destroy_node()
    rclpy.shutdown()

[INFO] [1753693039.835143470] [test_node]: TestNode has been started.
[INFO] [1753693039.887305329] [test_node]: Servo states: [ros_robot_controller_msgs.msg.BusServoState(present_id=[], target_id=[], position=[499], offset=[], voltage=[], temperature=[], position_limit=[], voltage_limit=[], max_temperature_limit=[], enable_torque=[], save_offset=[], stop=[]), ros_robot_controller_msgs.msg.BusServoState(present_id=[], target_id=[], position=[765], offset=[], voltage=[], temperature=[], position_limit=[], voltage_limit=[], max_temperature_limit=[], enable_torque=[], save_offset=[], stop=[]), ros_robot_controller_msgs.msg.BusServoState(present_id=[], target_id=[], position=[11], offset=[], voltage=[], temperature=[], position_limit=[], voltage_limit=[], max_temperature_limit=[], enable_torque=[], save_offset=[], stop=[]), ros_robot_controller_msgs.msg.BusServoState(present_id=[], target_id=[], position=[152], offset=[], voltage=[], temperature=[], position_limit=[], voltage_limit=[], max

ros_robot_controller_msgs.srv.GetBusServoState_Response(success=True, state=[ros_robot_controller_msgs.msg.BusServoState(present_id=[], target_id=[], position=[499], offset=[], voltage=[], temperature=[], position_limit=[], voltage_limit=[], max_temperature_limit=[], enable_torque=[], save_offset=[], stop=[]), ros_robot_controller_msgs.msg.BusServoState(present_id=[], target_id=[], position=[765], offset=[], voltage=[], temperature=[], position_limit=[], voltage_limit=[], max_temperature_limit=[], enable_torque=[], save_offset=[], stop=[]), ros_robot_controller_msgs.msg.BusServoState(present_id=[], target_id=[], position=[11], offset=[], voltage=[], temperature=[], position_limit=[], voltage_limit=[], max_temperature_limit=[], enable_torque=[], save_offset=[], stop=[]), ros_robot_controller_msgs.msg.BusServoState(present_id=[], target_id=[], position=[152], offset=[], voltage=[], temperature=[], position_limit=[], voltage_limit=[], max_temperature_limit=[], enable_torque=[], save_offse

[INFO] [1753693082.627714258] [test_node]: Останавливаем выполнение...
Failed to publish log message to rosout: publisher's context is invalid, at ./src/rcl/publisher.c:389


AttributeError: 'TestNode' object has no attribute 'stop_launch'

In [5]:


request = GetBusServoState.Request()
request.cmd = [GetBusServoCmd(id = i, get_position=1) for i in [1, 2, 3, 4, 5, 10]]
# self.get_logger().info('Updating servo positions...')        
if node.client.service_is_ready() == False:
    node.get_logger().error('Service is not ready')
future = node.client.call(request)


KeyboardInterrupt: 